In [25]:
import numpy as np                
import pandas as pd
import math
import matplotlib.pyplot as plt   
import time                       
import copy                        
from sympy import *               
init_printing()                   
plt.rcParams['figure.figsize'] = [10, 10] # Set plot size
ITERATIONS = 13 

In [26]:
wVal=np.array([[4.] ,[2.]])  
dVal=np.array([[4.] ,[3.]])

In [27]:
w=[]
d=[]
nVar=2
# Creating array symbols w and d in order to perform computation on functions using sympy
for i in range(len(wVal)):       
    w.append(symbols('w%d'%i))
    d.append(symbols('d%d'%i))
w=np.array(w)
d=np.array(d)
fun = w[0]**2*w[1]**2  # F(w) Function Initianization
fun.evalf()

In [28]:
def gradient(fun,w):                   
    res = []
    for i in range(len(w)):
        res.append([diff(fun,w[i])])   # Perform differentiation on the function fun in each variable of the array w
    return np.array(res)

In [29]:
phi=np.dot(gradient(fun,w).T,d) 
phi[0].evalf()

In [30]:
gradphi=gradient(phi,w)                  
g=gradient(fun,w)
gradphi=np.squeeze(gradphi, axis=1)      
print ("Hessian vector of F(w) in function of w and d:",gradphi)

Hessian vector of F(w) in function of w and d: [[2*d0*w1**2 + 4*d1*w0*w1]
 [4*d0*w0*w1 + 2*d1*w0**2]]


In [31]:
for i in range(len(w)):
    for j in range(len(w)):
        gradphi[i]= gradphi[i][0].subs(w[j], wVal[j]) 
        g[i]=g[i][0].subs(w[j], wVal[j]) 
print("Hessian vector of F(w) in function of d:",gradphi)
print ("Gradient F(w):",g)

Hessian vector of F(w) in function of d: [[2*d0*w1**2 + 4*d1*w0*w1]
 [4*d0*w0*w1 + 2*d1*w0**2]]
Gradient F(w): [[2*w0*w1**2]
 [2*w0**2*w1]]


In [32]:
def solveHessianVector(gradphi,dVal):
    res = copy.deepcopy(gradphi) # Using deepcopy to copy the variable by value and not reference
    d=[]
    for i in range(len(dVal)): #Filling symbols vector based on the number of parameters used
        d.append(symbols('d%d'%i))
    d=np.array(d)
    for i in range(len(dVal)):
        for j in range(len(dVal)):
            res[i][0]= res[i][0].subs(d[j], dVal[j])   #Substituting guess value vector value in the Hessian vector
    return res
solveHessianVector(gradphi,-g) 

array([[2*d0*w1**2 + 4*d1*w0*w1],
       [4*d0*w0*w1 + 2*d1*w0**2]], dtype=object)

In [33]:
def conjugateGradient(A,b,x):
    r = b - np.dot(A, x) 
    p=r 
    rOld = np.dot(r.T,r)
    for i in range(len(b)):     #Stop iteration corresponding to the size of b
        Ap = np.dot(A, p)
        alpha = (rOld/np.dot(p.T,Ap))
        x = x+alpha*p
        r = r - alpha*Ap
        rNew = np.dot(r.T,r)
        beta = rNew/rOld
        p = r + beta*p
        rOld = rNew
    return x
    
# Testing Conjugate Gradient
A = np.array([[4.,1.],[1.,3.]])
b = np.array([[1.],[2.]])
x = np.array([[2.],[1.]])
t1 = time.time()
print('start')
x=conjugateGradient(A,b,x)
t2 = time.time()
print("Computation time:",t2 - t1)
print ("x:",x)
print ("Ax=b:",np.dot(A,x))

start
Computation time: 0.0
x: [[0.09090909]
 [0.63636364]]
Ax=b: [[1.]
 [2.]]


In [62]:
def NewtonCG(gradphi,b,x):
    r = b - solveHessianVector(gradphi,x) #Substituting dot product with the Hessian vector using the guess values 
    p=r 
    rOld = np.dot(r.T,r)
    for i in range(len(b)):      #Stop iteration corresponding to the size of b, so of the gradient
        Ap = solveHessianVector(gradphi,p)
        alpha = (rOld/np.dot(p.T,Ap))
        x = x+alpha*p
        r = r - alpha*Ap
        rNew = np.dot(r.T,r)
        if sqrt(rNew) < 1e-10:
              break;
        beta = rNew/rOld
        p = r + beta*p
        rOld = rNew
    return x


In [50]:
def generateFun(nw): 
    w=[]
    x=symbols('x0')
    for i in range(nw): #Filling symbols vector based on the number of parameters used
        w.append(symbols('w%d'%i))
    w=np.array(w)
    fun=0
    #function generation 
    for i in range(0,nw):
        fun += w[i]*np.power(x, i)       
    return fun

#Testing created function
nw=4
fun = generateFun(nw)
fun

In [51]:
def generateDS(fun,wVal,size,noise):
    elem=[]
    y=[]
    clmn =[] 
    x=[]
    w=[]
    for i in range(len(wVal)):
        elem.append(np.linspace(-2, 2, size))
        clmn.append('x%d'%i)
        w.append(symbols('w%d'%i))
        x.append(symbols('x%d'%i))
    clmn.append('y')
    res=copy.deepcopy(fun)
    for i in range(size):
        for k in range(len(wVal)):
            res=res.subs(w[k], wVal[k])
            res=res.subs(x[k], elem[k][i])
        y.append(res +np.random.normal()*noise)
        res=copy.deepcopy(fun)
    y=np.squeeze(np.array(y))
    elem.append(y)
    elem=np.array(elem).T   
    df= pd.DataFrame(elem, columns=clmn).astype(float)
    return df

In [52]:
def solveFun(fun,sample,wVal):
    sumf = 0
    w=[]
    x=[]
    for i in range(len(wVal)):
        w.append(symbols('w%d'%i))
        x.append(symbols('x%d'%i))
    y=symbols('y')
    fun1=copy.deepcopy(fun)
    for i in range(len(sample)):
            for j in range(len(fun1)):
                for k in range(len(w)):
                    fun1[j][0]=fun1[j][0].subs(w[k], wVal[k][0])   #substituting values in the function
                    fun1[j][0]=fun1[j][0].subs(x[k], sample.iloc[i][k])
                fun1[j][0]=fun1[j][0].subs(y, sample.iloc[i][len(w)]) 
            sumf+=fun1
            fun1=copy.deepcopy(fun)
    sumf=sumf/(2.*len(sample))
    return sumf

In [53]:
def selectA(errfun,wVal,res,sampleFut,g):
    a=[0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1] # Possible learning rates
    n=0.01
    alpha=0.01
    sumF=0
    sumFP=0
    for i in range(len(a)):   
        wVal1=wVal+a[i]*res
        sumF=solveFun(errfun,sampleFut,wVal)    
        sumFP=solveFun(errfun,sampleFut,wVal1)
        if (sumFP<sumF+n*a[i]*np.dot(g.T,res)):
            alpha=a[i]
    return alpha

In [54]:
def subSampledHessianFree(errfun,df,wVal,empErr):
    #sh<s
    s=40
    sh=20
    dVal=[]
    for i in range(nw):
        dVal.append([1.])
    dVal=np.array(dVal)
    sample=df.sample(s)
    sampleh=df.sample(sh)
    d=[]
    w=[]
    x=[]
    for i in range(nw): #Filling symbols vector based on the number of parameters used
        d.append(symbols('d%d'%i))
        w.append(symbols('w%d'%i))
        x.append(symbols('x%d'%i))
    for i in range(ITERATIONS):
        g=gradient(errfun,w)
        phi=np.dot(g.T,d)        
        gradphi=gradient(phi,w) 
        gradphi=np.squeeze(gradphi, axis=1) #Vertical squeeze used for good function formatting
        sumg=solveFun(g,sample,wVal)
        sumgradphi = solveFun(gradphi,sampleh,wVal)
        dVal=NewtonCG(sumgradphi,-sumg,dVal)   #compute the conjugate gradient
        sample=df.sample(s) # select a random sample of the dataset of size s
        sampleh=df.sample(sh)
        lr= selectA(np.array([[errfun]]),wVal,dVal,sample,sumg)   #select the best learning rate
        wVal=wVal+lr*dVal
        empErr.append(np.squeeze(solveFun(np.array([[errfun]]),sample,wVal)))
    return wVal

In [55]:
def print2D(wValInit,wValFin,empErr):
    df.plot.scatter("x0","y",grid=True)
    xdf = pd.DataFrame(np.linspace(-2, 2, 20))
    ax = plt.axes()
    yfin = []
    yinit =[]
    w=[]
    x=[]
    for i in range(nw): #Filling symbols vector based on the number of parameters used
        w.append(symbols('w%d'%i))
        x.append(symbols('x%d'%i))
    w=np.array(w)
    x=np.array(x)
    res1=copy.deepcopy(fun)
    res2=copy.deepcopy(fun)
    for i in range (20):
        res1=res1.subs(x[0], xdf[0][i])
        res2=res2.subs(x[0], xdf[0][i])
        for j in range(nw):   
            res1=res1.subs(w[j], wValFin[j][0])
            res2=res2.subs(w[j], wValInit[j][0])
        yfin.append(res1)
        yinit.append(res2)
        res1=copy.deepcopy(fun)
        res2=copy.deepcopy(fun)

    ax.set_title('Model')
    ax.plot(xdf, yfin,color='r',linewidth=2.5)
    #ax.plot(xdf, yinit,color='b',linewidth=2.5)   
    fig1, ax2 = plt.subplots()
    xln = np.linspace(0, ITERATIONS, ITERATIONS)
    ax2 = plt.axes()    
    ax2.set_title('Empirical Risk')
    ax2.set_xlabel('Iterations')
    ax2.set_ylabel('Empirical Risk')
    plt.grid()
    ax2.plot(xln, empErr,linewidth=2.5,)

In [56]:
nw=2
size=400
wVal = np.random.randn(1,nw)[0]
wValInit = []
for i in range(nw):
        wValInit.append([40])
fun = generateFun(nw)
noise=1.5
df=generateDS(fun,wVal,size,noise)
y=symbols('y')
errfun=(fun -y)**2
print ("Generated function:",fun)
print ("Used value of W for generate the Dataset:",wVal)
print ("Error function:",errfun)

Generated function: w0 + w1*x0
Used value of W for generate the Dataset: [-0.37577004  0.04008667]
Error function: (w0 + w1*x0 - y)**2
